In [10]:
import numpy as np

# Creating a dataset
# Inputs [x1, x2]
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

# Labels for AND gate
y = np.array([0, 0, 0, 1])

# Initializing parameters
np.random.seed(42)  # for reproducibility
weights = np.random.rand(2)  # one weight per input
bias = np.random.rand(1)     # bias term
learning_rate = 0.1

# Activation function
def step_function(z):
    return int(z >= 0) if np.isscalar(z) or z.shape == () else np.where(z >= 0, 1, 0)

# Training loop
epochs = 20
for epoch in range(epochs):
    total_error = 0
    for xi, target in zip(X, y):
        # Forward pass
        z = np.dot(xi, weights) + bias
        output = step_function(z)

        # Error
        error = target - output
        total_error += abs(error)

        # Perceptron Learning Rule (Updating weights)
        weights += learning_rate * error * xi
        bias += learning_rate * error

    print(f"Epoch {epoch+1}/{epochs} - Total Error: {total_error}")

# Testing
print("\nFinal Weights:", weights)
print("Final Bias:", bias)

for xi in X:
    z = np.dot(xi, weights) + bias
    output = step_function(z)
    print(f"Input: {xi} => Output: {output}")

print("\nAccuracy:", sum((step_function(np.dot(x, weights) + bias) == y_i).item() for x, y_i in zip(X, y)), "/", len(y))

Epoch 1/20 - Total Error: [3]
Epoch 2/20 - Total Error: [3]
Epoch 3/20 - Total Error: [3]
Epoch 4/20 - Total Error: [1]
Epoch 5/20 - Total Error: [1]
Epoch 6/20 - Total Error: [2]
Epoch 7/20 - Total Error: [1]
Epoch 8/20 - Total Error: [0]
Epoch 9/20 - Total Error: [0]
Epoch 10/20 - Total Error: [0]
Epoch 11/20 - Total Error: [0]
Epoch 12/20 - Total Error: [0]
Epoch 13/20 - Total Error: [0]
Epoch 14/20 - Total Error: [0]
Epoch 15/20 - Total Error: [0]
Epoch 16/20 - Total Error: [0]
Epoch 17/20 - Total Error: [0]
Epoch 18/20 - Total Error: [0]
Epoch 19/20 - Total Error: [0]
Epoch 20/20 - Total Error: [0]

Final Weights: [0.17454012 0.35071431]
Final Bias: [-0.46800606]
Input: [0 0] => Output: [0]
Input: [0 1] => Output: [0]
Input: [1 0] => Output: [0]
Input: [1 1] => Output: [1]

Accuracy: 4 / 4


In [11]:
!pip -q install pennylane

import pennylane as qml
from pennylane import numpy as pnp

# Inputs [x1, x2]
X = pnp.array([
    [0., 0.],
    [0., 1.],
    [1., 0.],
    [1., 1.]
])

y = pnp.array([0., 0., 0., 1.])

# Quantum device
dev = qml.device("default.qubit", wires=1, shots=None)

# Single Qubit Model
@qml.qnode(dev)
def circuit(x, w):
    # Feature encoding (round 1)
    qml.RX(pnp.pi/2 * x[0], wires=0)
    qml.RY(pnp.pi/2 * x[1], wires=0)
    # Interaction term (x1 * x2) to help separate [1,0] vs [1,1]
    qml.RZ(pnp.pi/2 * (x[0]*x[1]), wires=0)

    # Trainable block 1
    qml.Rot(w[0], w[1], w[2], wires=0)

    # Feature encoding (round 2)
    qml.RX(pnp.pi/2 * x[0], wires=0)
    qml.RY(pnp.pi/2 * x[1], wires=0)

    # Trainable block 2
    qml.Rot(w[3], w[4], w[5], wires=0)

    return qml.expval(qml.PauliZ(0))

def model(x, w):
    # Map ⟨Z⟩ ∈ [-1,1] to probability ∈ [0,1]
    expval = circuit(x, w)
    return (1 - expval) / 2.0

# Binary Cross Entropy Loss
def loss(w):
    eps = 1e-7
    preds = pnp.array([model(xi, w) for xi in X])
    preds = pnp.clip(preds, eps, 1 - eps)
    return -pnp.mean(y * pnp.log(preds) + (1 - y) * pnp.log(1 - preds))

# Train
pnp.random.seed(7)
w = pnp.random.uniform(0, 2*pnp.pi, 6, requires_grad=True)
opt = qml.AdamOptimizer(stepsize=0.1)
epochs = 200

for epoch in range(1, epochs + 1):
    w, c = opt.step_and_cost(loss, w)
    if epoch % 20 == 0 or epoch == 1:
        print(f"Epoch {epoch:3d} - loss: {c:.6f}")

# Evaluation
def predict(x, w, threshold=0.5):
    p = float(model(x, w))
    return (1 if p >= threshold else 0), p

print("\nLearned parameters:", [float(a) for a in w])

correct = 0
for xi, yi in zip(X, y):
    cls, prob = predict(xi, w)
    correct += (cls == int(yi))
    print(f"Input {xi.astype(int)} -> prob(1)={prob:.3f} -> pred={cls} (target={int(yi)})")

print(f"\nAccuracy: {correct}/{len(X)}")

Epoch   1 - loss: 0.525762
Epoch  20 - loss: 0.209820
Epoch  40 - loss: 0.176702
Epoch  60 - loss: 0.174517
Epoch  80 - loss: 0.174259
Epoch 100 - loss: 0.174175
Epoch 120 - loss: 0.174174
Epoch 140 - loss: 0.174174
Epoch 160 - loss: 0.174173
Epoch 180 - loss: 0.174173
Epoch 200 - loss: 0.174173

Learned parameters: [-0.9113396581032276, 4.479204828296538, 4.052927885892655, 4.7123834823991375, 4.712365871730409, 3.3834693422771163]
Input [0 0] -> prob(1)=0.116 -> pred=0 (target=0)
Input [0 1] -> prob(1)=0.202 -> pred=0 (target=0)
Input [1 0] -> prob(1)=0.116 -> pred=0 (target=0)
Input [1 1] -> prob(1)=0.798 -> pred=1 (target=1)

Accuracy: 4/4
